In [ ]:
import skimage
import matplotlib.pyplot as plt
img_path="C:/Users/Admin/Desktop/datasets/lung.jpg"
img = skimage.io.imread(img_path)/255.0

def plotnoise(img, mode, r, c, i):
    plt.subplot(r,c,i)
    if mode is not None:
        gimg = skimage.util.random_noise(img, mode=mode)
        plt.imshow(gimg)
    else:
        plt.imshow(img)
    plt.title(mode)
    plt.axis("off")

plt.figure(figsize=(18,24))
r=4
c=4
plotnoise(img, "gaussian", r,c,1)
plotnoise(img, "localvar", r,c,2)
plotnoise(img, "poisson", r,c,3)
plotnoise(img, "salt", r,c,4)
plotnoise(img, "pepper", r,c,5)
plotnoise(img, "s&p", r,c,6)
plotnoise(img, "speckle", r,c,7)
# plotnoise(img, None, r,c,8)
plt.show()

In [ ]:
import cv2
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt

# Read the input image
img = cv2.imread("C:/Users/Admin/Desktop/datasets/original_image.png", cv2.IMREAD_GRAYSCALE)

# Reshape the image to a vector
data = img.reshape((-1, 1))

# Normalize the data to the range [0, 1]
data = data.astype(float) / 255.0

# Fuzzy c-means clustering
n_clusters = 3
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(data.T, n_clusters, 2, error=0.005, maxiter=1000, init=None)

# Get the cluster centers
cluster_centers = cntr

# Calculate the KL divergence-based fuzzy membership
def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

memberships = np.zeros((data.shape[0], n_clusters))
for i in range(data.shape[0]):
    for j in range(n_clusters):
        memberships[i, j] = kl_divergence(data[i], cluster_centers[j])

# Normalize the memberships
memberships = memberships / memberships.sum(axis=1, keepdims=True)

# Reshape the memberships to the original image shape
memberships = memberships.reshape(img.shape[0], img.shape[1], n_clusters)

# Select the cluster with the maximum membership for each pixel
segmented_img = np.argmax(memberships, axis=2)

# Display the segmented image
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img, cmap='gray')
plt.title('Original Image')
plt.subplot(1, 2, 2)
# Fuzzy c mean with kl divergence
plt.imshow(segmented_img, cmap='gray')
plt.title('Segmented Image')
plt.show()


In [ ]:
import numpy as np
import cv2

def pfcm(image, n_clusters, max_iter=1, m=2, alpha=1, error=1e-5):
    # Reshape the image to a vector
    data = image.reshape((-1, 1))
    
    # Normalize the data to the range [0, 1]
    data = data.astype(float) / 255.0
    
    # Number of pixels
    n_pixels = data.shape[0]
    
    # Initialize centroids randomly
    centroids = np.random.rand(n_clusters, data.shape[1])
    
    # Initialize membership, noise robustness, and spatial neighborhood matrices
    u = np.random.rand(n_pixels, n_clusters)
    eta = np.zeros((n_pixels, n_clusters))
    xi = np.zeros((n_pixels, n_clusters))
    
    for _ in range(max_iter):
        # Update membership
        for i in range(n_pixels):
            for j in range(n_clusters):
                u_sum = np.sum((np.linalg.norm(data[i] - centroids[j]) / np.linalg.norm(data[i] - centroids[k])) ** (1 / (m - 1)) for k in range(n_clusters))
                u[i, j] = (1 - eta[i, j] - xi[i, j]) / u_sum
        
        # Update centroids
        memberships = (u / (1 - eta - xi)) ** m
        centroids_new = np.dot(memberships.T, data) / np.sum(memberships, axis=0)[:, None]
        
        # Check for convergence
        if np.linalg.norm(centroids_new - centroids) < error:
            break
        
        centroids = centroids_new
    
        # Update noise robustness and spatial neighborhood matrices
        for i in range(n_pixels):
            for j in range(n_clusters):
                eta_sum = np.sum(xi[i, k] for k in range(n_clusters))
                eta[i, j] = 1 - xi[i, j] - ((n_clusters - 1) / n_clusters) * (eta_sum / np.sum((u[i, j] / u[i, k]) * ((np.linalg.norm(data[i] - centroids[j]) / np.linalg.norm(data[i] - centroids[k])) ** (1 / (m - 1))) for k in range(n_clusters)))
                xi[i, j] = 1 - (u[i, j] + eta[i, j]) - (1 - (u[i, j] + eta[i, j]) ** alpha) ** (1 / alpha)
    
    # Reshape the memberships to the original image shape
    memberships = u.reshape(image.shape[0], image.shape[1], n_clusters)
    
    # Select the cluster with the maximum membership for each pixel
    segmented_img = np.argmax(memberships, axis=2)
    
    return segmented_img

# Read the input image
image = cv2.imread("C:/Users/Admin/Desktop/datasets/original_image.png", cv2.IMREAD_GRAYSCALE)

# Apply PFCM clustering
n_clusters = 3
segmented_image = pfcm(image, n_clusters)

# Display the segmented image
plt.imshow(segmented_image, cmap='gray')
plt.title('Segmented Image')
# plt.colorbar()
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def add_gaussian_noise(image, mean=0, std=5):
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image

# Image path
img_path = "C:/Users/Admin/Desktop/datasets/lung.jpg"

# Load the image
image = cv2.imread(img_path)

# Add Gaussian noise to the image
noisy_image = add_gaussian_noise(image)

# Plot the original and noisy images
plt.figure(figsize=(5, 5))

# plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')

# plt.subplot(1, 2, 2)


plt.show()


# Gaussian Noise

In [ ]:
plt.imshow(cv2.cvtColor(noisy_image, cv2.COLOR_BGR2RGB))
plt.title('Noisy Image')
plt.axis('off')
plt.show()

In [ ]:
# Apply median filter to remove noise
denoised_image = cv2.medianBlur(noisy_image, 5)
plt.imshow(cv2.cvtColor(denoised_image, cv2.COLOR_BGR2RGB))
plt.title('Denoised Image')
plt.axis('off')

# Salt and pepper noise

In [ ]:
def add_salt_and_pepper_noise(image, salt_ratio=0.05, pepper_ratio=0.05):
    """
    Adds salt and pepper noise to an image.

    Args:
        image (numpy.ndarray): Input image.
        salt_ratio (float): Ratio of salt noise (default: 0.05).
        pepper_ratio (float): Ratio of pepper noise (default: 0.05).

    Returns:
        numpy.ndarray: Image with salt and pepper noise.
    """
    row, col, _ = image.shape
    salt = np.random.rand(row, col) < salt_ratio
    pepper = np.random.rand(row, col) < pepper_ratio
    noisy_image = np.copy(image)
    noisy_image[salt] = 1
    noisy_image[pepper] = 4
    return noisy_image


# Add salt and pepper noise
spimage = add_salt_and_pepper_noise(image)
plt.imshow(cv2.cvtColor(spimage, cv2.COLOR_BGR2RGB))
plt.title('Noisy Image')
plt.axis('off')
plt.show()

# Impulse Noise

In [ ]:
def add_impulse_noise(image, intensity=20):
    noisy_image = np.copy(image)
    num_pixels = int(image.size * intensity / 100)  # Calculate the number of pixels based on intensity percentage
    pixels_to_change = np.random.choice(image.size, num_pixels, replace=False)
    noisy_image.flat[pixels_to_change] = np.random.randint(0, 256, num_pixels)
    return noisy_image
# Add impulse noise to the image
noisy_image_impulse = add_impulse_noise(image, intensity=20)
plt.imshow(cv2.cvtColor(noisy_image_impulse, cv2.COLOR_BGR2RGB))
plt.title('Noisy Image (Impulse)')
plt.axis('off')
plt.show()


# Speckle Noise

In [ ]:
def add_speckle_noise(image, mean=0, std=0.5):
    """
    Add speckle noise to an image.
    
    Args:
        image (numpy.ndarray): Input image.
        mean (float): Mean of the noise (default: 0).
        std (float): Standard deviation of the noise (default: 0.1).
    
    Returns:
        numpy.ndarray: Image with speckle noise.
    """
    row, col, ch = image.shape
    gauss = np.random.normal(mean, std, (row, col, ch))
    noisy_image = image + image * gauss
    return noisy_image
# Convert the input image to a suitable data type (e.g., 8-bit unsigned integer)
# Convert the input image to a suitable data type (e.g., 8-bit unsigned integer)
noisy_image_speckle = add_speckle_noise(image)
image_uint8 = cv2.convertScaleAbs(noisy_image_speckle)

# Display the noisy image with speckle noise
plt.imshow(cv2.cvtColor(image_uint8, cv2.COLOR_BGR2RGB))
# plt.title('Speckle Noise')
plt.axis('off')
plt.show()


# Rician Noise

In [ ]:
def add_rician_noise(image, mean=0, std=2):
    """
    Add Rician noise to an image.
    
    Args:
        image (numpy.ndarray): Input image.
        mean (float): Mean of the Gaussian noise (default: 0).
        std (float): Standard deviation of the Gaussian noise (default: 0.1).
    
    Returns:
        numpy.ndarray: Image with Rician noise.
    """
    row, col, ch = image.shape
    X = np.random.normal(mean, std, (row, col, ch))
    Y = np.random.normal(mean, std, (row, col, ch))
    rician_noise = np.sqrt(X**2 + Y**2)
    noisy_image = image + rician_noise
    return noisy_image
noisy_image_rician = add_rician_noise(image)

noisy_image_normalized = noisy_image_rician / np.max(noisy_image_rician)
plt.imshow(noisy_image_normalized)


In [ ]:
plt.imshow(cv2.cvtColor(noisy_image, cv2.COLOR_BGR2RGB))
plt.title('Noisy Image')
plt.axis('off')
plt.show()

In [ ]:
# Apply Gaussian blur to remove noise
blurred_image = cv2.GaussianBlur(noisy_image, (5, 5), 0)


plt.imshow(blurred_image)
plt.title('Blurred Image (Noise Removed)')
plt.axis('off')

# Fuzzy c means clustering


In [ ]:

import skfuzzy as fuzz

def fuzzy_c_means_clustering(data, n_clusters, m=2, max_iter=100, error=0.005):
    """
    Perform Fuzzy C-Means clustering on the input data.

    Parameters:
        data (ndarray): Input data, where each row corresponds to a data point.
        n_clusters (int): Number of clusters.
        m (float, optional): Fuzzifier exponent. Defaults to 2.
        max_iter (int, optional): Maximum number of iterations. Defaults to 100.
        error (float, optional): Stopping criterion. Defaults to 0.005.

    Returns:
        ndarray: Cluster centers.
        ndarray: Membership values of data points to each cluster.
    """
    # Perform Fuzzy C-Means clustering
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(data.T, n_clusters, m,
                                                      error=error, maxiter=max_iter,
                                                      init=None)
    return cntr, u


# Convert the noisy image to grayscale
gray_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2GRAY)

# Reshape the grayscale image into a 1D array
data = gray_image.reshape((-1, 1))

# Define parameters for Fuzzy C-Means clustering
n_clusters = 5
max_iter = 100
m = 2
error = 0.005

# Fuzzy C-Means clustering
cluster_centers, membership_values = fuzzy_c_means_clustering(data, n_clusters, m, max_iter, error)

# Get the clustered image
clustered_image = np.uint8(np.round(cluster_centers[membership_values.argmax(axis=0)]))
clustered_image = clustered_image.reshape(gray_image.shape)

plt.imshow(clustered_image, cmap='gray')
# plt.title('Fuzzy C means')
plt.axis('off')

plt.show()


In [ ]:
from skimage.metrics import peak_signal_noise_ratio
import cv2

original_img = cv2.imread("C:/Users/Admin/Desktop/datasets/lung.jpg")
segmented_img = cv2.imread("C:/Users/Admin/Desktop/datasets/resultimg/fcm.png")

# Resize segmented image to match original image dimensions
segmented_img_resized = cv2.resize(segmented_img, (original_img.shape[1], original_img.shape[0]))

# Calculate PSNR value
psnr_value = peak_signal_noise_ratio(original_img, segmented_img_resized)

print("PSNR value:", psnr_value)


In [ ]:
from math import log10, sqrt 
import cv2 
import numpy as np 

def PSNR(original, compressed): 
    original_height, original_width, _ = original.shape
    compressed = cv2.resize(compressed, (original_width, original_height))
    
    mse = np.mean((original - compressed) ** 2) 
    if mse == 0: 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 

def main(): 
    original = cv2.imread("C:/Users/Admin/Desktop/datasets/lung.jpg") 
    compressed = cv2.imread("C:/Users/Admin/Desktop/datasets/resultimg/fcm.png", 1) 
    value = PSNR(original, compressed) 
    print(f"PSNR value is {value} dB") 
    
if __name__ == "__main__": 
    main() 


In [ ]:
import cv2
import numpy as np
import math

def psnr(original_img, compressed_img):
    original_height, original_width, _ = original_img.shape
    compressed_img = cv2.resize(compressed_img, (original_width, original_height))
    
    original_img = original_img.astype(np.float32)
    compressed_img = compressed_img.astype(np.float32)
    
    mse = np.mean((original_img - compressed_img) ** 2)
    
    max_pixel = 255.0
    if mse == 0:
        return float('inf')
    psnr_value = 10 * math.log10((max_pixel ** 2) / mse)
    return psnr_value

# Example usage:
original_image = cv2.imread("C:/Users/Admin/Desktop/datasets/lung.jpg") 
compressed_image = cv2.imread("C:/Users/Admin/Desktop/datasets/resultimg/fcm.png")

psnr_value = psnr(original_image, compressed_image)
print("PSNR value:", psnr_value, "dB")


In [ ]:
import cv2
import numpy as np

def misclassification_error_rate(original_img, segmented_img):
    # Convert images to grayscale
    original_gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    segmented_gray = cv2.cvtColor(segmented_img, cv2.COLOR_BGR2GRAY)
    
    # Threshold the segmented image to obtain binary mask
    _, segmented_mask = cv2.threshold(segmented_gray, 0, 255, cv2.THRESH_BINARY)
    
    # Resize segmented mask to match the dimensions of the original grayscale image
    segmented_mask_resized = cv2.resize(segmented_mask, (original_gray.shape[1], original_gray.shape[0]))
    
    # Calculate misclassification error rate
    intersection = cv2.bitwise_and(original_gray, segmented_mask_resized)
    union = cv2.bitwise_or(original_gray, segmented_mask_resized)
    misclassification_error_rate = 1.0 - np.sum(intersection) / np.sum(union)
    
    return misclassification_error_rate * 100


# Calculate misclassification error rate
error_rate = misclassification_error_rate(original_img, segmented_img)
print("Misclassification Error Rate:", error_rate)


In [ ]:
import cv2
import numpy as np

def psnr(original_img, segmented_img):
    # Resize segmented image to match the dimensions of the original image
    segmented_img_resized = cv2.resize(segmented_img, (original_img.shape[1], original_img.shape[0]))

    # Ensure images are of type float
    original_img = original_img.astype(float)
    segmented_img_resized = segmented_img_resized.astype(float)

    # Calculate MSE
    mse = np.mean((original_img - segmented_img_resized) ** 2)

    # Calculate maximum possible pixel value
    max_pixel_value = np.max(original_img)

    # Calculate PSNR
    psnr_value = 10 * np.log10((max_pixel_value ** 2) / mse)

    return psnr_value


psnr_value = psnr(original_img, segmented_img)
print("PSNR:", psnr_value)


In [ ]:
import cv2
import numpy as np

def calculate_accuracy(original_img, segmented_img):
    # Convert images to grayscale
    original_gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    segmented_gray = cv2.cvtColor(segmented_img, cv2.COLOR_BGR2GRAY)
    
    # Threshold the segmented image to obtain binary mask
    _, segmented_mask = cv2.threshold(segmented_gray, 0, 255, cv2.THRESH_BINARY)
    
    # Resize segmented mask to match the dimensions of the original grayscale image
    segmented_mask_resized = cv2.resize(segmented_mask, (original_gray.shape[1], original_gray.shape[0]))
    
    # Define positive and negative regions based on the segmented mask
    positive_regions = original_gray > 0  # Regions with non-zero intensity in the original image
    negative_regions = original_gray == 0  # Background regions in the original image
    
    # Count the number of pixels classified as positive or negative in the segmented mask
    positive_pixels_count = cv2.countNonZero(segmented_mask_resized)
    negative_pixels_count = segmented_mask_resized.size - positive_pixels_count
    
    # Count the number of true positives, true negatives, false positives, and false negatives
    TP = np.sum(np.logical_and(positive_regions, segmented_mask_resized))
    TN = np.sum(np.logical_and(negative_regions, np.logical_not(segmented_mask_resized)))
    FP = positive_pixels_count - TP
    FN = negative_pixels_count - TN
    
    # Calculate the accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    return accuracy


# Calculate accuracy
accuracy = calculate_accuracy(original_img, segmented_img)
print("Accuracy:", accuracy)


In [ ]:
import cv2
import numpy as np

def calculate_sensitivity(original_img, segmented_img):
    # Convert images to grayscale
    original_gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    segmented_gray = cv2.cvtColor(segmented_img, cv2.COLOR_BGR2GRAY)
    
    # Threshold the segmented image to obtain binary mask
    _, segmented_mask = cv2.threshold(segmented_gray, 0, 255, cv2.THRESH_BINARY)
    
    # Resize segmented mask to match the dimensions of the original grayscale image
    segmented_mask_resized = cv2.resize(segmented_mask, (original_gray.shape[1], original_gray.shape[0]))
    
    # Calculate True Positives (TP) and False Negatives (FN)
    TP = np.sum(np.logical_and(original_gray > 0, segmented_mask_resized > 0))
    FN = np.sum(np.logical_and(original_gray > 0, segmented_mask_resized == 0))
    
    # Calculate Sensitivity (Sen)
    sensitivity = TP / (TP + FN)
    
    return sensitivity

# Calculate Sensitivity
sensitivity = calculate_sensitivity(original_img, segmented_img)
print("Sensitivity (Sen):", sensitivity)


In [ ]:
# import cv2
# import numpy as np

# def psnr(img1, img2):
#     mse = np.mean((img1 - img2) ** 2)
#     if mse == 0:
#         return float('inf')
#     max_pixel = 255.0
#     psnr_val = 20 * np.log10(max_pixel / np.sqrt(mse))
#     return psnr_val

# # def psnr_segmented(original_img, segmented_img):
# #     # Resize segmented image to match original image dimensions
# #     segmented_img_resized = cv2.resize(segmented_img, (original_img.shape[1], original_img.shape[0]))

# #     segmented_regions = cv2.split(segmented_img_resized)
# #     original_regions = cv2.split(original_img)
# #     psnr_values = []
# #     for seg_region, orig_region in zip(segmented_regions, original_regions):
# #         psnr_val = psnr(seg_region, orig_region)
# #         psnr_values.append(psnr_val)
# #     return psnr_values

# # Load original image and segmented image
# original_img = cv2.imread("C:/Users/Admin/Desktop/datasets/lung.jpg")
# segmented_img = cv2.imread("C:/Users/Admin/Desktop/datasets/fc1.png")

# # Compute PSNR for each segmented region
# psnr_values = psnr_segmented(original_img, segmented_img)

# # Print PSNR values for each segmented region
# for i, psnr_val in enumerate(psnr_values):
#     print(f"PSNR for segmented region {i+1}: {psnr_val:.2f} dB")


In [ ]:
# print(clustered_image)

# Fuzzy C Means clustering With KL Divergence

In [ ]:
def ifcm_image_segmentations(clustered_image, num_clusters=4, m=2, h=0.1, lamda=1.0, max_iters=100, error_threshold=1e-4):
    num_samples, num_features = clustered_image.shape
    cluster_centers = np.random.rand(num_clusters, num_features)
    u = np.random.rand(num_samples, num_clusters)
    v = np.random.rand(num_samples, num_clusters)
    hesitancy = np.full((num_samples, num_clusters), h)

    for iteration in range(max_iters):
        for j in range(num_clusters):
            numerator = np.sum((u[:, j] ** m + (1 - v[:, j]) ** m)[:, np.newaxis] * clustered_image, axis=0)
            denominator = np.sum((u[:, j] ** m + (1 - v[:, j]) ** m))
            cluster_centers[j] = numerator / denominator
        # ... (rest of your code)

        # Initialize Lagrange multipliers
        lagrange_multipliers = np.zeros((num_samples, num_clusters))

        # Update membership degrees (u_ij) and Lagrange multipliers using Lagrange's method
        for i in range(num_samples):
            for j in range(num_clusters):
                u_ij_star = np.exp(-np.linalg.norm(clustered_image[i] - cluster_centers[j]) ** 2 / lamda)
                u_sum = np.sum(np.exp(-np.linalg.norm(clustered_image[i] - cluster_centers[k]) ** 2  / lamda) for k in range(num_clusters))
                u[i, j] = u_ij_star / u_sum

                # Calculate the Lagrange multiplier term
                lagrange_term = lamda * np.sum(u[i, k] * np.log(u[i, k] / v[i, k]) for k in range(num_clusters))
                lagrange_multipliers[i, j] = lagrange_term

        # Check for convergence
        if np.linalg.norm(u - v) < error_threshold:
            break

        v = u.copy()  # Update v with the current u
    

    # After clustering is complete, reshape the results back to the original shape
    cluster_assignments = np.argmax(u, axis=1)
    results = clustered_image.reshape(image.shape[:2])

    return results, cluster_assignments, v

result_images, cluster_assignments, non_membership_matrix = ifcm_image_segmentations(clustered_image, num_clusters=4, m=2, h=0.1)


In [ ]:
# results=result_image.reshape(resized_image.shape)
# plt.subplot(142)
plt.imshow(result_images, cmap='gray')
print(result_images)
# plt.title('Segmentation')
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
from skimage.metrics import peak_signal_noise_ratio
import cv2

original_img = cv2.imread("C:/Users/Admin/Desktop/datasets/lung.jpg")
segmented_img = cv2.imread("C:/Users/Admin/Desktop/datasets/resultimg/fcmkl.png")

# Resize segmented image to match original image dimensions
segmented_img_resized = cv2.resize(segmented_img, (original_img.shape[1], original_img.shape[0]))

# Calculate PSNR value
psnr_value = peak_signal_noise_ratio(original_img, segmented_img_resized)

print("PSNR value:", psnr_value)


def misclassification_error_rate(original_img, segmented_img):
    # Convert images to grayscale
    original_gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    segmented_gray = cv2.cvtColor(segmented_img, cv2.COLOR_BGR2GRAY)
    
    # Threshold the segmented image to obtain binary mask
    _, segmented_mask = cv2.threshold(segmented_gray, 0, 255, cv2.THRESH_BINARY)
    
    # Resize segmented mask to match the dimensions of the original grayscale image
    segmented_mask_resized = cv2.resize(segmented_mask, (original_gray.shape[1], original_gray.shape[0]))
    
    # Calculate misclassification error rate
    intersection = cv2.bitwise_and(original_gray, segmented_mask_resized)
    union = cv2.bitwise_or(original_gray, segmented_mask_resized)
    misclassification_error_rate = 1.0 - np.sum(intersection) / np.sum(union)
    
    return misclassification_error_rate * 100



# Calculate misclassification error rate
error_rate = misclassification_error_rate(original_img, segmented_img)
print("Misclassification Error Rate:", error_rate)


def calculate_accuracy(original_img, segmented_img):
    # Convert images to grayscale
    original_gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    segmented_gray = cv2.cvtColor(segmented_img, cv2.COLOR_BGR2GRAY)
    
    # Threshold the segmented image to obtain binary mask
    _, segmented_mask = cv2.threshold(segmented_gray, 0, 255, cv2.THRESH_BINARY)
    
    # Resize segmented mask to match the dimensions of the original grayscale image
    segmented_mask_resized = cv2.resize(segmented_mask, (original_gray.shape[1], original_gray.shape[0]))
    
    # Define positive and negative regions based on the segmented mask
    positive_regions = original_gray > 0  # Regions with non-zero intensity in the original image
    negative_regions = original_gray == 0  # Background regions in the original image
    
    # Count the number of pixels classified as positive or negative in the segmented mask
    positive_pixels_count = cv2.countNonZero(segmented_mask_resized)
    negative_pixels_count = segmented_mask_resized.size - positive_pixels_count
    
    # Count the number of true positives, true negatives, false positives, and false negatives
    TP = np.sum(np.logical_and(positive_regions, segmented_mask_resized))
    TN = np.sum(np.logical_and(negative_regions, np.logical_not(segmented_mask_resized)))
    FP = positive_pixels_count - TP
    FN = negative_pixels_count - TN
    
    # Calculate the accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    return accuracy



# Calculate accuracy
accuracy = calculate_accuracy(original_img, segmented_img)
print("Accuracy:", accuracy)


def calculate_sensitivity(original_img, segmented_img):
    # Convert images to grayscale
    original_gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    segmented_gray = cv2.cvtColor(segmented_img, cv2.COLOR_BGR2GRAY)
    
    # Threshold the segmented image to obtain binary mask
    _, segmented_mask = cv2.threshold(segmented_gray, 0, 255, cv2.THRESH_BINARY)
    
    # Resize segmented mask to match the dimensions of the original grayscale image
    segmented_mask_resized = cv2.resize(segmented_mask, (original_gray.shape[1], original_gray.shape[0]))
    
    # Calculate True Positives (TP) and False Negatives (FN)
    TP = np.sum(np.logical_and(original_gray > 0, segmented_mask_resized > 0))
    FN = np.sum(np.logical_and(original_gray > 0, segmented_mask_resized == 0))
    
    # Calculate Sensitivity (Sen)
    sensitivity = TP / (TP + FN)
    
    return sensitivity



# Calculate Sensitivity
sensitivity = calculate_sensitivity(original_img, segmented_img)
print("Sensitivity (Sen):", sensitivity)


# Picture Fuzzy C-Means Clustering

In [ ]:

# Define kernel function (Gaussian kernel)
def kernel_function(x1, x2, sigma=10):
    return np.exp(-np.sum((x1 - x2)**2) / ( (sigma**2)))


# Define parameters
n_clusters = 5
max_iterations = 2
tolerance = 0.001
m = 2
alpha = 10

# Initialize cluster centers randomly
centers = np.random.rand(n_clusters, data.shape[1])

# Initialize membership matrix randomly
membership_matrix = np.random.rand(n_clusters, len(data))

# Fuzzy C-Means clustering
for _ in range(max_iterations):
    # Update eta and xi parameters
    for i in range(len(data)):
        for j in range(n_clusters):
            # Initialize eta_ij and xi_ij
            xi_ij = 1
            eta_ij = 1

            # Compute xi_ij
            xi_ij = 1 - (membership_matrix[j, i] + eta_ij) - ((1 - (membership_matrix[j, i] + eta_ij)) ** alpha) ** (1 / alpha)

            # Compute eta_ij
            denominator_eta = np.sum((membership_matrix[j, i] / membership_matrix[k, i]) * ((np.linalg.norm(data[i] - centers[j]) ** 2) / (np.linalg.norm(data[i] - centers[k]) ** 2)) ** (1 / (m - 1)) for k in range(n_clusters))
            eta_ij = 1 - xi_ij - ((n_clusters - 1) / n_clusters) * np.sum(xi_ij for k in range(n_clusters)) / denominator_eta

            # Update membership matrix
            numerator_membership = (1 - eta_ij - xi_ij)
            denominator_membership = np.sum(((np.linalg.norm(data[i] - centers[j]) ** 2) / (np.linalg.norm(data[i] - centers[k]) ** 2)) ** (1 / (m - 1)) for k in range(n_clusters))
            membership_matrix[j, i] = numerator_membership / denominator_membership

    # Update cluster centers
    for j in range(n_clusters):
        numerator_centers = np.sum((membership_matrix[j, i] / (1 - eta_ij - xi_ij)) ** m * data[i] for i in range(len(data)))
        denominator_centers = np.sum((membership_matrix[j, i] / (1 - eta_ij - xi_ij)) ** m for i in range(len(data)))
        centers[j] = numerator_centers / denominator_centers


# Extract cluster labels
cluster_labels = np.argmax(membership_matrix, axis=0)

# Reshape the cluster labels back to the shape of the original image
clustered_images = cluster_labels.reshape(result_images.shape)

# Display the clustered image
plt.imshow(clustered_images, cmap='gray')
plt.title('Picture Fuzzy C-Means Clustering')
plt.axis('off')
plt.show()


# kernel fuzzy clustering based on picture fuzzy sets and KL divergence

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Define kernel function (Gaussian kernel)
def kernel_function(x1, x2, sigma=10):
    return np.exp(-np.sum((x1 - x2)**2) / (2 * (sigma**2)))

# Define the robust semi-supervised picture fuzzy clustering function
def picture_fuzzy_clustering(data_points, n_clusters, max_iterations, m, alpha, gamma, sigma):
    n_points = len(data_points)
    n_features = data_points.shape[1]

    # Initialize cluster centers randomly
    centers = np.random.rand(n_clusters, n_features)

    # Initialize membership matrix randomly
    membership_matrix = np.random.rand(n_clusters, n_points)

    for _ in range(max_iterations):
        # Update membership matrix
        membership_matrix, eta_matrix, xi_matrix = update_membership_matrix(data_points, centers, membership_matrix, m, alpha, gamma)

        # Update cluster centers
        centers = update_cluster_centers(data_points, membership_matrix, eta_matrix, xi_matrix, sigma)

    return centers, membership_matrix

# Define function to update membership matrix, eta, and xi
def update_membership_matrix(data_points, centers, membership_matrix, m, alpha, gamma):
    n_clusters = centers.shape[0]
    n_points = len(data_points)

    updated_membership_matrix = np.zeros_like(membership_matrix)
    eta_matrix = np.zeros_like(membership_matrix)
    xi_matrix = np.zeros_like(membership_matrix)

    for i in range(n_points):
        for j in range(n_clusters):
            # Calculate pi_ij
            pi_ij = np.mean(membership_matrix[j] * (2 - xi_matrix))

            # Calculate mu_ij
            mu_numerator = pi_ij * np.exp(-(1 - data_points[i] - centers[j]) / gamma)
            mu_denominator = np.sum([np.mean(membership_matrix[l] * (2 - xi_matrix)) * np.exp(-(1 - data_points[i] - centers[l]) / gamma) for l in range(n_clusters)])
            mu_ij = mu_numerator / mu_denominator * (1 / (2 - xi_matrix[j, i]))

            # Calculate k(x_j, v_i)
            k_xj_vi = np.exp(-np.linalg.norm(data_points[i] - centers[j])**2 / (2 * (sigma**2)))

            # Calculate eta_ij
            exp_xi_ij = np.exp(-xi_matrix[j, i])
            sum_exp_xi_lj = np.sum(np.exp(-xi_matrix[l, i]) for l in range(n_clusters))
            eta_matrix[j, i] = exp_xi_ij / sum_exp_xi_lj * (1 - np.sum(xi_matrix[:, i]) / n_clusters)

            # Calculate xi_ij
            u_ij = updated_membership_matrix[j, i]
            xi_matrix[j, i] = 1 - (u_ij + eta_matrix[j, i]) - (1 - (u_ij + eta_matrix[j, i])**alpha)**(1/alpha)

            # Update membership matrix elements
            updated_membership_matrix[j, i] = mu_ij * (2 - xi_matrix[j, i]) * (1 - k_xj_vi)

    return updated_membership_matrix, eta_matrix, xi_matrix


# Define function to update cluster centers
def update_cluster_centers(data_points, membership_matrix, eta_matrix, xi_matrix, sigma):
    n_clusters = membership_matrix.shape[0]
    n_points = len(data_points)
    n_features = data_points.shape[1]

    centers = np.zeros((n_clusters, n_features))

    for j in range(n_clusters):
        numerator = np.zeros(n_features)
        denominator = 0

        for i in range(n_points):
            mu_xi = membership_matrix[j, i] * (2 - xi_matrix[j, i])
            k_xj_vi = np.exp(-np.linalg.norm(data_points[i] - centers[j])**2 / (2 * (sigma**2)))

            numerator += mu_xi * k_xj_vi * data_points[i]
            denominator += mu_xi * k_xj_vi

        centers[j] = numerator / denominator

    return centers

# Load the image
# image = cv2.imread('/content/drive/MyDrive/y0.jpg')
# gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Reshape the image into a 1D array of data points
data_points = clustered_images.reshape((-1, 1)).astype(np.float32)

# Define parameters
n_clusters = 2
max_iterations = 10
m = 3
alpha = 1
gamma = 1
sigma = 10

# Perform robust semi-supervised picture fuzzy clustering
centers, membership_matrix = picture_fuzzy_clustering(data_points, n_clusters, max_iterations, m, alpha, gamma, sigma)

# Reshape the cluster labels back to the shape of the original image
cluster_labels = np.argmax(membership_matrix, axis=0)
clustere_img= cluster_labels.reshape(clustered_images.shape)

# Display the clustered image
plt.imshow(clustere_img, cmap='gray')
plt.title('Robust Semi-Supervised Picture Fuzzy Clustering')
plt.axis('off')
plt.show()


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage import io
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from sklearn.metrics import confusion_matrix
from skfuzzy import cmeans

# Load the input image
image_path = "C:/Users/Admin/Desktop/datasets/lung.jpg"
image = io.imread(image_path)

# Convert the image to grayscale
gray_image = rgb2gray(image)

# Apply Otsu's thresholding to obtain a binary image
threshold_value = threshold_otsu(gray_image)
binary_image = gray_image > threshold_value

# Flatten the binary image
data = binary_image.flatten()

# Perform Fuzzy C-means clustering
centers, u, _, _, _, _, _ = cmeans(data.reshape(1, -1), c=2, m=2, error=0.005, maxiter=1000, seed=0)

# Get the cluster labels
cluster_labels = np.argmax(u, axis=0)

# Reshape the cluster labels to the original image shape
cluster_labels = cluster_labels.reshape(binary_image.shape)

# Display the original image and the clustered image
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(binary_image, cmap='gray')
plt.title('Binary Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cluster_labels, cmap='gray')
plt.title('Clustered Image')
plt.axis('off')

plt.tight_layout()
plt.show()

# Assuming you have ground truth labels for the image
# Create ground truth labels (for demonstration purposes)
ground_truth_labels = np.zeros_like(cluster_labels)
ground_truth_labels[50:150, 50:150] = 1  # Assuming the object is located in this region

# Calculate confusion matrix
conf_matrix = confusion_matrix(ground_truth_labels.flatten(), cluster_labels.flatten())
print("Confusion Matrix:")
print(conf_matrix)


In [ ]:
# Load the input image
image_path = "C:/Users/Admin/Desktop/datasets/lung.jpg"
image = io.imread(image_path)

# Assuming the lung region is located in a specific area of the image
lung_region = np.zeros_like(image)
# lung_region[100:300, 200:400] = 1  # Assuming the lung region is located in this region

# Display the lung region
plt.imshow(lung_region, cmap='gray')
plt.title('Ground Truth Labels (Lung Region)')
plt.axis('off')
plt.show()


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage import io
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from sklearn.metrics import confusion_matrix
from skfuzzy import cmeans

def add_gaussian_noise(image, mean=0, std=5):
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image

# Load the input image
image_path = "C:/Users/Admin/Desktop/datasets/lung.jpg"
image = io.imread(image_path)

# Convert the image to grayscale
gray_image = rgb2gray(image)

# Apply Otsu's thresholding to obtain a binary image
threshold_value = threshold_otsu(gray_image)
binary_image = gray_image > threshold_value

# Add Gaussian noise to the original image
noisy_image = add_gaussian_noise(image)

# Convert the noisy image to grayscale
noisy_gray_image = rgb2gray(noisy_image)

# Apply Otsu's thresholding to obtain a binary image from the noisy image
threshold_value_noisy = threshold_otsu(noisy_gray_image)
binary_image_noisy = noisy_gray_image > threshold_value_noisy

# Flatten the binary image from the noisy image
data_noisy = binary_image_noisy.flatten()

# Perform Fuzzy C-means clustering on the noisy image
centers_noisy, u_noisy, _, _, _, _, _ = cmeans(data_noisy.reshape(1, -1), c=2, m=2, error=0.005, maxiter=1000, seed=0)

# Get the cluster labels for the noisy image
cluster_labels_noisy = np.argmax(u_noisy, axis=0)

# Reshape the cluster labels to the original image shape for the noisy image
cluster_labels_noisy = cluster_labels_noisy.reshape(binary_image_noisy.shape)

# Display the original image, noisy image, and the clustered image of the noisy image
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(image)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(noisy_image)
plt.title('Noisy Image')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(cluster_labels_noisy, cmap='gray')
plt.title('Clustered Image (Noisy)')
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Generate artificial ground truth labels (for demonstration purposes)
ground_truth_labels = np.zeros_like(cluster_labels_noisy)
ground_truth_labels[50:150, 50:150] = 1  # Assuming the object is located in this region

# Calculate confusion matrix
conf_matrix = confusion_matrix(ground_truth_labels.flatten(), cluster_labels_noisy.flatten())
print("Confusion Matrix:")
print(conf_matrix)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the original image
original_img = cv2.imread("C:/Users/Admin/Desktop/datasets/lung.jpg")

# Preprocessing (if needed)

# Convert the image to grayscale
gray_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)

# Thresholding: Simple binary segmentation (lung vs. background)
_, binary_mask = cv2.threshold(gray_img, 100, 255, cv2.THRESH_BINARY)



# Post-processing (if needed)

# Visualization
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(original_img[:, :, ::-1])  # Convert BGR to RGB for display
plt.title('Original Image')
print(original_img.shape)
plt.axis('off')

plt.show()


In [ ]:

plt.imshow(binary_mask, cmap='gray')
# print(binary_mask.shape)
# print(binary_mask)
# plt.title('Ground Truth Segmentation (Binary)')
plt.axis('off')

In [ ]:
def add_gaussian_noise(image, mean=0, std=3):
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image

In [ ]:

# Add Gaussian noise to the image
noisy_image = add_gaussian_noise(binary_mask)

plt.imshow(cv2.cvtColor(noisy_image, cv2.COLOR_BGR2RGB))
print(noisy_image.shape)
# plt.title('Gaussian noisy_image')
plt.axis('off')

plt.show()

In [ ]:
# Apply Gaussian blur to remove noise
blurred_image = cv2.GaussianBlur(noisy_image, (5, 5), 0)


plt.imshow(blurred_image,cmap='gray')
# plt.title('Blurred Image (Noise Removed)')
plt.axis('off')

# Fuzzy c means clustering

In [ ]:
class FCM:
    def __init__(self, n_clusters=2, max_iter=100, m=2, error=1e-5, random_state=None):
        self.u, self.centers = None, None
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.m = m
        self.error = error
        self.random_state = random_state

    def fit(self, X):
      n_samples = X.shape[0]
      if self.random_state is not None:
        np.random.seed(self.random_state)
      u = np.random.rand(n_samples, self.n_clusters)
      u = u / np.sum(u, axis=1, keepdims=True)

      for _ in range(self.max_iter):
        u_old = u.copy()
        centers = self._calculate_centers(X, u)
        u = self._update_membership(X, centers)
        if np.linalg.norm(u - u_old.reshape(u.shape)) < self.error:
            break

      self.u = u
      self.centers = centers



    def _calculate_centers(self, X, u):
        um = u ** self.m
        centers = um.T.dot(X) / np.sum(um, axis=0, keepdims=True).T
        return centers

    def _update_membership(self, X, centers):
        dist = np.linalg.norm(X[:, np.newaxis] - centers, axis=2)
        dist = np.fmax(dist, np.finfo(np.float64).eps)
        u = dist[:, :, np.newaxis] / dist[:, :, np.newaxis].sum(axis=1, keepdims=True)
        u = u ** (2 / (self.m - 1))
        u = u / u.sum(axis=1, keepdims=True)
        return u

    def predict(self, X):
        dist = np.linalg.norm(X[:, np.newaxis] - self.centers, axis=2)
        return np.argmin(dist, axis=1)
    
  # Convert the input image to 2D array and normalize
image_2d = blurred_image.reshape((-1, 1)).astype(float) / 255.0

# Perform Fuzzy C-means clustering
n_clusters = 5  # Number of clusters
max_iter = 100  # Maximum number of iterations
fcm = FCM(n_clusters=n_clusters, max_iter=max_iter)
fcm.fit(image_2d)

# Predict cluster labels
cluster_labels = fcm.predict(image_2d)

# Reshape cluster labels to the shape of the original image
cluster_labels_2d = cluster_labels.reshape(blurred_image.shape[0], blurred_image.shape[1])

plt.imshow(cluster_labels_2d, cmap='gray')
print(cluster_labels_2d)
# plt.title('Segmented Image')
plt.axis('off')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
# Load the ground truth input image
ground_truth_img = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png", cv2.IMREAD_GRAYSCALE)
# Resize the ground truth image to match the shape of cluster_membership_image
ground_truth_img = cv2.resize(ground_truth_img, (cluster_labels_2d.shape[1], cluster_labels_2d.shape[0]))
# Calculate confusion matrix
conf_matrix = confusion_matrix(ground_truth_img.flatten(), cluster_labels_2d.flatten())

# print("Confusion Matrix:")
# print(conf_matrix)
# Calculate true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]
TP = conf_matrix[1, 1]

# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Calculate sensitivity (true positive rate or recall)
sensitivity = TP / (TP + FN)
precision = TP / (TP + FP)
print("TP=",TP)
print('TN=',TN)
print('FN=',FN)
print('FP=',FP)
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Precision:", precision)



 
from math import log10, sqrt 
 
def PSNR(original, compressed): 
    compressed_resized = cv2.resize(compressed, (original.shape[1], original.shape[0]))
    mse = np.mean((original - compressed_resized) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 
  
def main(): 
     original = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png") 
     compressed = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c1.png", 1) 
     value = PSNR(original, compressed) 
     print(f"PSNR value is {value} dB") 
       
if __name__ == "__main__": 
    main() 



from skimage import metrics
# Load images
image1 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png")
image2 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c1.png")
image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
# print(image1.shape, image2.shape)
# Convert images to grayscale
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Calculate SSIM
ssim_score = metrics.structural_similarity(image1_gray, image2_gray, full=True)
print(f"SSIM Score: ", round(ssim_score[0], 2))
from sklearn.metrics import jaccard_score
import numpy as np

# Assuming image1_gray and image2_gray are binary masks (0 or 255)
binary_mask1 = (image1_gray > 0).astype(int)
binary_mask2 = (image2_gray > 0).astype(int)

# Calculate the Jaccard Index
JI = jaccard_score(binary_mask1.flatten(), binary_mask2.flatten())
print("Jaccard Index:", JI)
import numpy as np

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    dice = (2.0 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice

# Example usage:
dice_score = dice_coefficient(image1_gray.astype(bool), image2_gray.astype(bool))
print("Dice Score:", dice_score)


In [ ]:
from sklearn.metrics import confusion_matrix
# Load the ground truth input image
ground_truth_img = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png", cv2.IMREAD_GRAYSCALE)
# Resize the ground truth image to match the shape of cluster_membership_image
ground_truth_img = cv2.resize(ground_truth_img, (cluster_labels_2d.shape[1], cluster_labels_2d.shape[0]))
# Calculate confusion matrix
conf_matrix = confusion_matrix(ground_truth_img.flatten(), cluster_labels_2d.flatten())

# print("Confusion Matrix:")
# print(conf_matrix)
# Calculate true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]
TP = conf_matrix[1, 1]

# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Calculate sensitivity (true positive rate or recall)
sensitivity = TP / (TP + FN)
precision = TP / (TP + FP)
print("TP=",TP)
print('TN=',TN)
print('FN=',FN)
print('FP=',FP)
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Precision:", precision)


In [ ]:
 
from math import log10, sqrt 
 
def PSNR(original, compressed): 
    compressed_resized = cv2.resize(compressed, (original.shape[1], original.shape[0]))
    mse = np.mean((original - compressed_resized) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 
  
def main(): 
     original = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png") 
     compressed = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c1.png", 1) 
     value = PSNR(original, compressed) 
     print(f"PSNR value is {value} dB") 
       
if __name__ == "__main__": 
    main() 

In [ ]:
from skimage import metrics
# Load images
image1 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png")
image2 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c1.png")
image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
# print(image1.shape, image2.shape)
# Convert images to grayscale
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Calculate SSIM
ssim_score = metrics.structural_similarity(image1_gray, image2_gray, full=True)
print(f"SSIM Score: ", round(ssim_score[0], 2))

In [ ]:
from sklearn.metrics import jaccard_score
import numpy as np

# Assuming image1_gray and image2_gray are binary masks (0 or 255)
binary_mask1 = (image1_gray > 0).astype(int)
binary_mask2 = (image2_gray > 0).astype(int)

# Calculate the Jaccard Index
JI = jaccard_score(binary_mask1.flatten(), binary_mask2.flatten())
print("Jaccard Index:", JI)


In [ ]:
import numpy as np

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    dice = (2.0 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice

# Example usage:
dice_score = dice_coefficient(image1_gray.astype(bool), image2_gray.astype(bool))
print("Dice Score:", dice_score)


# Fuzzy c means clustering with KL divergence

In [ ]:
import cv2
import numpy as np

def likl_segmentation(blurred_image, num_clusters=4, m=2, lamda=1.0, max_iters=100, error_threshold=1e-4):
    # Load the image
    image = blurred_image.astype(np.float32)
    
    # Convert the image to float32
    image = np.float32(image)
    
    # Initialize cluster centers randomly
    num_samples, num_features = image.shape
    cluster_centers = np.random.rand(num_clusters, num_features)
    
    # Initialize membership degrees randomly
    u = np.random.rand(num_samples, num_clusters)
    
    # Initialize Lagrange multipliers
    lagrange_multipliers = np.zeros((num_samples, num_clusters))
    
    u_old = np.zeros_like(u)  # Initialize u_old outside the loop
    
    for iteration in range(max_iters):
        # Update cluster centers
        for j in range(num_clusters):
            numerator = np.sum((u[:, j] ** m)[:, np.newaxis] * image, axis=0)
            denominator = np.sum(u[:, j] ** m)
            cluster_centers[j] = numerator / denominator
        
        # Update membership degrees and Lagrange multipliers
        for i in range(num_samples):
            for j in range(num_clusters):
                u_ij_star = np.exp(-np.linalg.norm(image[i] - cluster_centers[j]) ** 2 / lamda)
                u_sum = np.sum(np.exp(-np.linalg.norm(image[i] - cluster_centers[k]) ** 2 / lamda) for k in range(num_clusters))
                u[i, j] = u_ij_star / u_sum
                
                # Calculate the Lagrange multiplier term
                lagrange_term = lamda * np.sum(u[i, k] * np.log(u[i, k]) for k in range(num_clusters))
                lagrange_multipliers[i, j] = lagrange_term
        
        # Check for convergence
        if np.linalg.norm(u - u_old) < error_threshold:
            break
        
        u_old = u.copy()  # Update u_old with the current u
    
    # After clustering is complete, reshape the results back to the original shape
    cluster_assignments = np.argmax(u, axis=1)
    results = image.reshape((num_samples, num_features))  # Fix the reshape
    
    return results

# segmented_image = likl_segmentation(noisy_image)
# Perform further processing or visualization with the segmented_image


In [ ]:

segmented_image = likl_segmentation(blurred_image)

# Display the segmented image
plt.imshow(segmented_image, cmap='gray')  # You can use any colormap you prefer
# plt.title('Segmented Image')
plt.axis('off')
# plt.colorbar()
plt.show()

In [ ]:
print(segmented_image.shape)
print(segmented_image)

In [ ]:
from sklearn.metrics import confusion_matrix
# Load the ground truth input image
ground_truth_img = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png", cv2.IMREAD_GRAYSCALE)
# Resize the ground truth image to match the shape of cluster_membership_image
ground_truth_img = cv2.resize(ground_truth_img, (segmented_image.shape[1], segmented_image.shape[0]))
# Calculate confusion matrix
conf_matrix = confusion_matrix(ground_truth_img.flatten(), segmented_image.flatten())

# print("Confusion Matrix:")
# print(conf_matrix)
# Calculate true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]
TP = conf_matrix[1, 1]

# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Calculate sensitivity (true positive rate or recall)
sensitivity = TP / (TP + FN)
precision = TP / (TP + FP)
print("TP=",TP)
print('TN=',TN)
print('FN=',FN)
print('FP=',FP)
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Precision:", precision)



 
from math import log10, sqrt 
 
def PSNR(original, compressed): 
    compressed_resized = cv2.resize(compressed, (original.shape[1], original.shape[0]))
    mse = np.mean((original - compressed_resized) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 
  
def main(): 
     original = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png") 
     compressed = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c2.png", 1) 
     value = PSNR(original, compressed) 
     print(f"PSNR value is {value} dB") 
       
if __name__ == "__main__": 
    main() 



from skimage import metrics
# Load images
image1 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png")
image2 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c2.png")
image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
# print(image1.shape, image2.shape)
# Convert images to grayscale
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Calculate SSIM
ssim_score = metrics.structural_similarity(image1_gray, image2_gray, full=True)
print(f"SSIM Score: ", round(ssim_score[0], 2))
from sklearn.metrics import jaccard_score
import numpy as np

# Assuming image1_gray and image2_gray are binary masks (0 or 255)
binary_mask1 = (image1_gray > 0).astype(int)
binary_mask2 = (image2_gray > 0).astype(int)

# Calculate the Jaccard Index
JI = jaccard_score(binary_mask1.flatten(), binary_mask2.flatten())
print("Jaccard Index:", JI)
import numpy as np

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    dice = (2.0 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice

# Example usage:
dice_score = dice_coefficient(image1_gray.astype(bool), image2_gray.astype(bool))
print("Dice Score:", dice_score)


# Picture fuzzy c means clustering


In [ ]:
print(blurred_image.shape)


In [ ]:
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from skimage.metrics import peak_signal_noise_ratio

class FCMCustom:
    def __init__(self, n_clusters=2, max_iter=100, sigma=1.0, alpha=1.0, gamma=1.0):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.sigma = sigma
        self.alpha = alpha
        self.gamma = gamma
        self.u = None

    def fit(self, X):
        n_samples = X.shape[0]
        n_features = X.shape[1]
        centers = np.random.rand(self.n_clusters, n_features)
        eta = np.random.rand(n_samples, self.n_clusters)
        xi = np.random.rand(n_samples, self.n_clusters)
        self.u = np.random.rand(n_samples, self.n_clusters)

        for _ in range(self.max_iter):
            w = self._calculate_w(X)
            eta = self._calculate_eta(xi)
            xi = self._calculate_xi(X, centers, eta)
            centers = self._calculate_centers(X, w)

        self.u = self._calculate_u(X, centers, eta, xi, w)
        self.centers = centers

    def _calculate_w(self, X):
        c = self.n_clusters
        n_samples = X.shape[0]
        w = np.zeros((n_samples, c))

        for i in range(n_samples):
            for j in range(c):
                w[i, j] = np.exp(-np.sum((self.u[:, j] - self.u[i, j]) ** 2) / self.sigma)
            w[i] /= np.sum(w[i])
        return w

    def _calculate_eta(self, xi):
        return (xi ** 2) / (self.n_clusters * (xi + 2 * xi))

    def _calculate_xi(self, X, centers, eta):
        alpha = self.alpha
        xi = np.zeros_like(eta)
        for i in range(X.shape[0]):
            for j in range(self.n_clusters):
                xi[i, j] = 1 - (self.u[i, j] + eta[i, j]) - (1 - (self.u[i, j] + eta[i, j]) ** alpha) ** (1 / alpha)
        return xi

    def _calculate_centers(self, X, w):
        c = self.n_clusters
        centers = np.zeros((c, X.shape[1]))
        for j in range(c):
            denominator = np.sum(w[:, j])
            if denominator != 0:  # Avoid division by zero
                numerator = np.sum((w[:, j][:, np.newaxis] * X), axis=0)
                centers[j] = numerator / denominator
            else:
                centers[j] = 0  # Set centers to zero if any denominator is zero
        return centers

    def _calculate_u(self, X, centers, eta, xi, w):
      n_samples = X.shape[0]
      c = self.n_clusters
      u = np.zeros((n_samples, c))
      for i in range(n_samples):
        for j in range(c):
            denominator = 1 - eta[i, j] - xi[i, j]
            if not np.isclose(denominator, 0):  # Avoid division by zero
                numerator = w[i, j] * np.exp(-np.sum((X[i] - centers[j]) ** 2))
                u[i, j] = numerator / denominator
            else:
                u[i, j] = 0  # Set membership to zero if any denominator is close to zero
        u[i] /= np.sum(u[i])
      return u


    def predict(self, X):
        dist = np.linalg.norm(X[:, np.newaxis] - self.centers, axis=2)
        return np.argmin(dist, axis=1)

# Read the original image
image_path = "C:/Users/Admin/Desktop/datasets/Ground_Img/dam.png"
original_image = io.imread(image_path)

# Reshape the image data to a 2D array
height, width, channels = original_image.shape
image_2d = original_image.reshape((height * width, channels))

# Normalize the image data
image_2d = image_2d.astype(float) / 255.0

# Perform Fuzzy C-means clustering
n_clusters = 2  # Number of clusters
max_iter = 1  # Maximum number of iterations
sigma = 70  # Sigma parameter
alpha = 1.0  # Alpha parameter
gamma = 1.0  # Gamma parameter
fcm_custom = FCMCustom(n_clusters=n_clusters, max_iter=max_iter, sigma=sigma, alpha=alpha, gamma=gamma)
fcm_custom.fit(image_2d)

# Predict cluster labels
cluster_labels = fcm_custom.predict(image_2d)

# Reshape cluster labels to the shape of the original image
cluster_labels_2 = cluster_labels.reshape((height, width))

# Plot the segmented image
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(original_image)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cluster_labels_2, cmap='gray')
#Picture fuzzy c means clusturing
print(original_image.shape)
print(cluster_labels_2.shape)
plt.title(" Picture fuzzy c means clusturing")
plt.axis('off')

plt.show()


In [ ]:
plt.imshow(original_image)
# plt.title('Original Image')
plt.axis('off')

plt.show()

In [ ]:
plt.imshow(cluster_labels_2, cmap='gray')
#Picture fuzzy c means clusturing
print(original_image.shape)
print(cluster_labels_2.shape)
# plt.title(" Picture fuzzy c means clusturing")
plt.axis('off')

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
# Load the ground truth input image
ground_truth_img = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png", cv2.IMREAD_GRAYSCALE)
# Resize the ground truth image to match the shape of cluster_membership_image
ground_truth_img = cv2.resize(ground_truth_img, (cluster_labels_2.shape[1], cluster_labels_2.shape[0]))
# Calculate confusion matrix
conf_matrix = confusion_matrix(ground_truth_img.flatten(), cluster_labels_2.flatten())

# print("Confusion Matrix:")
# print(conf_matrix)
# Calculate true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]
TP = conf_matrix[1, 1]

# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Calculate sensitivity (true positive rate or recall)
sensitivity = TP / (TP + FN)
precision = TP / (TP + FP)
print("TP=",TP)
print('TN=',TN)
print('FN=',FN)
print('FP=',FP)
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Precision:", precision)



 
from math import log10, sqrt 
 
def PSNR(original, compressed): 
    compressed_resized = cv2.resize(compressed, (original.shape[1], original.shape[0]))
    mse = np.mean((original - compressed_resized) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 
  
def main(): 
     original = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png") 
     compressed = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c4.png", 1) 
     value = PSNR(original, compressed) 
     print(f"PSNR value is {value} dB") 
       
if __name__ == "__main__": 
    main() 



from skimage import metrics
# Load images
image1 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png")
image2 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c4.png")
image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
# print(image1.shape, image2.shape)
# Convert images to grayscale
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Calculate SSIM
ssim_score = metrics.structural_similarity(image1_gray, image2_gray, full=True)
print(f"SSIM Score: ", round(ssim_score[0], 2))
from sklearn.metrics import jaccard_score
import numpy as np

# Assuming image1_gray and image2_gray are binary masks (0 or 255)
binary_mask1 = (image1_gray > 0).astype(int)
binary_mask2 = (image2_gray > 0).astype(int)

# Calculate the Jaccard Index
JI = jaccard_score(binary_mask1.flatten(), binary_mask2.flatten())
print("Jaccard Index:", JI)
import numpy as np

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    dice = (2.0 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice

# Example usage:
dice_score = dice_coefficient(image1_gray.astype(bool), image2_gray.astype(bool))
print("Dice Score:", dice_score)


# kernel fuzzy clustering based on picture fuzzy sets and KL divergence

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Define kernel function (Gaussian kernel)
def kernel_function(x1, x2, sigma=1):
    return np.exp(-np.sum((x1 - x2)**2) / (2 * (sigma**2)))

# Define the robust semi-supervised picture fuzzy clustering function
def picture_fuzzy_clustering(data_points, n_clusters, max_iterations, m, alpha, gamma, sigma):
    n_points = len(data_points)
    n_features = data_points.shape[1]

    # Initialize cluster centers randomly
    centers = np.random.rand(n_clusters, n_features)

    # Initialize membership matrix randomly
    membership_matrix = np.random.rand(n_clusters, n_points)

    for _ in range(max_iterations):
        # Update membership matrix
        membership_matrix, eta_matrix, xi_matrix = update_membership_matrix(data_points, centers, membership_matrix, m, alpha, gamma)

        # Update cluster centers
        centers = update_cluster_centers(data_points, membership_matrix, eta_matrix, xi_matrix, sigma)

    return centers, membership_matrix

# Define function to update membership matrix, eta, and xi
def update_membership_matrix(data_points, centers, membership_matrix, m, alpha, gamma):
    n_clusters = centers.shape[0]
    n_points = len(data_points)

    updated_membership_matrix = np.zeros_like(membership_matrix)
    eta_matrix = np.zeros_like(membership_matrix)
    xi_matrix = np.zeros_like(membership_matrix)

    for i in range(n_points):
        for j in range(n_clusters):
            # Calculate pi_ij
            pi_ij = np.mean(membership_matrix[j] * (2 - xi_matrix))

            # Calculate mu_ij
            mu_numerator = pi_ij * np.exp(-(1 - data_points[i] - centers[j]) / gamma)
            mu_denominator = np.sum([np.mean(membership_matrix[l] * (2 - xi_matrix)) * np.exp(-(1 - data_points[i] - centers[l]) / gamma) for l in range(n_clusters)])
            mu_ij = mu_numerator / mu_denominator * (1 / (2 - xi_matrix[j, i]))

            # Calculate k(x_j, v_i)
            k_xj_vi = np.exp(-np.linalg.norm(data_points[i] - centers[j])**2 / (2 * (sigma**2)))

            # Calculate eta_ij
            exp_xi_ij = np.exp(-xi_matrix[j, i])
            sum_exp_xi_lj = np.sum(np.exp(-xi_matrix[l, i]) for l in range(n_clusters))
            eta_matrix[j, i] = exp_xi_ij / sum_exp_xi_lj * (1 - np.sum(xi_matrix[:, i]) / n_clusters)

            # Calculate xi_ij
            u_ij = updated_membership_matrix[j, i]
            xi_matrix[j, i] = 1 - (u_ij + eta_matrix[j, i]) - (1 - (u_ij + eta_matrix[j, i])**alpha)**(1/alpha)

            # Update membership matrix elements
            updated_membership_matrix[j, i] = mu_ij * (2 - xi_matrix[j, i]) * (1 - k_xj_vi)

    return updated_membership_matrix, eta_matrix, xi_matrix


# Define function to update cluster centers
def update_cluster_centers(data_points, membership_matrix, eta_matrix, xi_matrix, sigma):
    n_clusters = membership_matrix.shape[0]
    n_points = len(data_points)
    n_features = data_points.shape[1]

    centers = np.zeros((n_clusters, n_features))

    for j in range(n_clusters):
        numerator = np.zeros(n_features)
        denominator = 0

        for i in range(n_points):
            mu_xi = membership_matrix[j, i] * (2 - xi_matrix[j, i])
            k_xj_vi = np.exp(-np.linalg.norm(data_points[i] - centers[j])**2 / (2 * (sigma**2)))

            numerator += mu_xi * k_xj_vi * data_points[i]
            denominator += mu_xi * k_xj_vi

        centers[j] = numerator / denominator

    return centers

# Load the image
image = cv2.imread('C:/Users/Admin/Desktop/datasets/Ground_Img/dam.png')

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

print(gray_image.shape)

# Reshape the image into a 1D array of data points
data_points = gray_image.reshape((-1, 1)).astype(np.float32)

# Define parameters
n_clusters = 2
max_iterations = 1
m = 3
alpha = 1
gamma = 1
sigma = 1

# Perform robust semi-supervised picture fuzzy clustering
centers, membership_matrix = picture_fuzzy_clustering(data_points, n_clusters, max_iterations, m, alpha, gamma, sigma)

# Reshape the cluster labels back to the shape of the original image
cluster_labels = np.argmax(membership_matrix, axis=0)
clustered_image = cluster_labels.reshape(gray_image.shape)

# Display the clustered image
plt.imshow(clustered_image, cmap='gray')
# plt.title('Robust Semi-Supervised Picture Fuzzy Clustering')
plt.axis('off')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
# Load the ground truth input image
ground_truth_img = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png", cv2.IMREAD_GRAYSCALE)
# Resize the ground truth image to match the shape of cluster_membership_image
ground_truth_img = cv2.resize(ground_truth_img, (clustered_image.shape[1], clustered_image.shape[0]))
# Calculate confusion matrix
conf_matrix = confusion_matrix(ground_truth_img.flatten(), clustered_image.flatten())

# print("Confusion Matrix:")
# print(conf_matrix)
# Calculate true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]
TP = conf_matrix[1, 1]

# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Calculate sensitivity (true positive rate or recall)
sensitivity = TP / (TP + FN)
precision = TP / (TP + FP)
print("TP=",TP)
print('TN=',TN)
print('FN=',FN)
print('FP=',FP)
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Precision:", precision)

from math import log10, sqrt 
 
def PSNR(original, compressed): 
    compressed_resized = cv2.resize(compressed, (original.shape[1], original.shape[0]))
    mse = np.mean((original - compressed_resized) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 
  
def main(): 
     original = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png") 
     compressed = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c5.png", 1) 
     value = PSNR(original, compressed) 
     print(f"PSNR value is {value} dB") 
       
if __name__ == "__main__": 
    main() 



from skimage import metrics
# Load images
image1 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png")
image2 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c5.png")
image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
# print(image1.shape, image2.shape)
# Convert images to grayscale
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Calculate SSIM
ssim_score = metrics.structural_similarity(image1_gray, image2_gray, full=True)
print(f"SSIM Score: ", round(ssim_score[0], 2))
from sklearn.metrics import jaccard_score
import numpy as np

# Assuming image1_gray and image2_gray are binary masks (0 or 255)
binary_mask1 = (image1_gray > 0).astype(int)
binary_mask2 = (image2_gray > 0).astype(int)

# Calculate the Jaccard Index
JI = jaccard_score(binary_mask1.flatten(), binary_mask2.flatten())
print("Jaccard Index:", JI)
import numpy as np

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    dice = (2.0 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice

# Example usage:
dice_score = dice_coefficient(image1_gray.astype(bool), image2_gray.astype(bool))
print("Dice Score:", dice_score)


# proposed work

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz

# Define kernel function (Gaussian kernel)
def kernel_function(x1, x2, sigma=70):
    return np.exp(-np.sum((x1 - x2)**2) / ( (sigma**2)))

# Load the image
image = cv2.imread('C:/Users/Admin/Desktop/datasets/Ground_Img/dam.png')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Reshape the image into a 1D array of data points
data_points = gray_image.reshape((-1, 1)).astype(np.float32)

# Define parameters
n_clusters = 2
max_iterations = 2
tolerance = 0.001
m = 2
alpha = 10
lambda_val = 10

# Initialize cluster centers randomly
centers = np.random.rand(n_clusters, data_points.shape[1])

# Initialize membership matrix randomly
membership_matrix = np.random.rand(n_clusters, len(data_points))

# Fuzzy C-Means clustering
for _ in range(max_iterations):
    # Compute distance matrix (using Gaussian kernel)
    distance_matrix = np.zeros((n_clusters, len(data_points)))
    for j in range(n_clusters):
        for i in range(len(data_points)):
            distance_matrix[j, i] = kernel_function(data_points[i], centers[j])

    # Update membership matrix
    for j in range(n_clusters):
        for i in range(len(data_points)):
            numerator = np.exp((-lambda_val + 2 * (1 + alpha) * (1 - distance_matrix[j, i])) * (alpha * np.sum(distance_matrix[:, i] * membership_matrix[:, j]) - membership_matrix[j, i]))
            denominator = np.sum([np.exp((-lambda_val + 2 * (1 + alpha) * (1 - distance_matrix[k, i])) * (alpha * np.sum(distance_matrix[:, i] * membership_matrix[:, k]) - membership_matrix[k, i])) for k in range(n_clusters)])
            membership_matrix[j, i] = numerator / denominator

    # Update cluster centers
    for j in range(n_clusters):
        numerator = np.sum([(membership_matrix[j, i] * kernel_function(data_points[i], centers[j]) * data_points[i]) + (alpha * (membership_matrix[j, i] - distance_matrix[j, i]) ** 2 * data_points[i]) for i in range(len(data_points))], axis=0)
        denominator = np.sum([membership_matrix[j, i] * kernel_function(data_points[i], centers[j]) + (alpha * (membership_matrix[j, i] - distance_matrix[j, i]) ** 2) for i in range(len(data_points))])
        centers[j] = numerator / denominator

# Extract cluster labels
cluster_labels = np.argmax(membership_matrix, axis=0)

# Reshape the cluster labels back to the shape of the original image
clustered_images = cluster_labels.reshape(gray_image.shape)

# Display the clustered image
plt.imshow(clustered_images, cmap='gray')
print(clustered_images)
# plt.title('Fuzzy C-Means Clustering')
plt.axis('off')
plt.show()


In [ ]:
# plt.imshow(clustered_images, cmap='gray')
# print(clustered_images)
# # plt.title('Fuzzy C-Means Clustering')
# plt.axis('off')
# plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
# Load the ground truth input image
ground_truth_img = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png", cv2.IMREAD_GRAYSCALE)
# Resize the ground truth image to match the shape of cluster_membership_image
ground_truth_img = cv2.resize(ground_truth_img, (clustered_images.shape[1], clustered_images.shape[0]))
# Calculate confusion matrix
conf_matrix = confusion_matrix(ground_truth_img.flatten(), clustered_images.flatten())

# print("Confusion Matrix:")
# print(conf_matrix)
# Calculate true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]
TP = conf_matrix[1, 1]

# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Calculate sensitivity (true positive rate or recall)
sensitivity = TP / (TP + FN)
precision = TP / (TP + FP)
print("TP=",TP)
print('TN=',TN)
print('FN=',FN)
print('FP=',FP)
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Precision:", precision)



 
from math import log10, sqrt 
 
def PSNR(original, compressed): 
    compressed_resized = cv2.resize(compressed, (original.shape[1], original.shape[0]))
    mse = np.mean((original - compressed_resized) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 
  
def main(): 
     original = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png") 
     compressed = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c6.png", 1) 
     value = PSNR(original, compressed) 
     print(f"PSNR value is {value} dB") 
       
if __name__ == "__main__": 
    main() 



from skimage import metrics
# Load images
image1 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/1.png")
image2 = cv2.imread("C:/Users/Admin/Desktop/datasets/Ground_Img/c6.png")
image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
# print(image1.shape, image2.shape)
# Convert images to grayscale
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Calculate SSIM
ssim_score = metrics.structural_similarity(image1_gray, image2_gray, full=True)
print(f"SSIM Score: ", round(ssim_score[0], 2))
from sklearn.metrics import jaccard_score
import numpy as np

# Assuming image1_gray and image2_gray are binary masks (0 or 255)
binary_mask1 = (image1_gray > 0).astype(int)
binary_mask2 = (image2_gray > 0).astype(int)

# Calculate the Jaccard Index
JI = jaccard_score(binary_mask1.flatten(), binary_mask2.flatten())
print("Jaccard Index:", JI)
import numpy as np

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    dice = (2.0 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice

# Example usage:
dice_score = dice_coefficient(image1_gray.astype(bool), image2_gray.astype(bool))
print("Dice Score:", dice_score)
